In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

# Sellers

🎯 Our goal is to find sellers who repeatedly underperform vs others, and understand why.  

👍 This will help us shape our recommendations about how to improve Olist's profit margin for the future.

🔥 Notebook best practices 👇
<details>
    <summary> - <i>Click here</i> - </summary>

From now on, exploratory notebooks are going to get pretty long, and we strongly advise you to follow these notebook principles
- Code your logic so that your Notebook can always be run from top to bottom without crashing (`Cell --> Run All`)
- Name your variables carefully 
- Use dummy names such as `tmp` for intermediary steps when you know you won't need them later
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you're between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





## 1 - Code `olist/seller.py` 

👉 Create the method `get_training_data` in `olist/seller.py` that will return a DataFrame with the following features:
  
| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `seller_id` 	| str 	| the id of the seller **UNIQUE** 	|
| `seller_city` 	| str 	| the city where seller is located 	|
| `seller_state` 	| str 	| the state where seller is located 	|
| `delay_to_carrier` 	| float 	| returns 0 if the order is delivered before the shipping_limit_date, otherwise the value of the delay 	|
| `wait_time` 	| float 	| average wait_time (duration of deliveries) per seller 	|
| `date_first_sale` 	| datetime 	| date of the first sale on Olist 	|
| `date_last_sale` 	| datetime 	| date of the last sale on Olist 	|
| `share_of_five_stars` 	| float 	| share of five-star reviews for orders in which the seller was involved 	|
| `share_of_one_stars` 	| float 	| share of one-star reviews for orders in which the seller was involved 	|
| `review_score` 	| float 	| average review score for orders in which the seller was involved 	|
| `n_orders` 	| int 	| number of unique orders the seller was involved with 	|
| `quantity` 	| int 	| total number of items sold by this seller 	|
| `quantity_per_order` 	| float 	| average number of items per order for this seller 	|
| `sales` 	| float 	| total sales associated with this seller (excluding freight value) in BRL 	|  
  
💡 As usual, use the notebook to draft your different functions, and once you are satisfied with their outputs, copy-paste them to `olist/seller.py`

### `get_seller_features`
    returns a DataFrame with: 'seller_id', 'seller_city', 'seller_state'

In [ ]:
# YOUR CODE HERE

### `get_seller_delay_wait_time`
    returns a DataFrame with: 'seller_id', 'delay_to_carrier', 'wait_time'

In [ ]:
# YOUR CODE HERE

### `get_active_dates`
    returns a DataFrame with 'seller_id', 'date_first_sale', 'date_last_sale'

In [ ]:
# YOUR CODE HERE

### `get_review_score`
    returns a DataFrame with 'seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score'

In [ ]:
# YOUR CODE HERE

### `get_quantity`
    returns a DataFrame with 'seller_id', 'n_orders', 'quantity', 'quantity_per_order'

> ❌ Here, you cannot start from the `matching_table`.  Why ?<br/>  ⚠️ If you do so and group by sellers, the number of orders will be inflated by the number of reviews because one order can receive multiple reviews... ! <br/> ✅ The easiest way to workaround this issue is simply to start from `order_items`

In [ ]:
# YOUR CODE HERE

### `get_sales`
    return 'seller_id', 'sales'

In [ ]:
# YOUR CODE HERE

### 💾 Save your logic into `seller.py`

### Test your code

In [2]:
from nbresult import ChallengeResult
from olist.seller import Seller
data = Seller().get_training_data()

result = ChallengeResult('seller',
    shape=data.shape,
    columns=sorted(list(data.columns)),
    avg_review_score=int(data['review_score'].mean()),
    unique_state=sorted(list(data['seller_state'].unique())),
    min_wait_time=round(data['wait_time'].min(), 2),
    max_wait_time=int(data['wait_time'].max()),
    avg_wait_time=int(data['wait_time'].mean()),
    avg_delay_carrier=round(data['delay_to_carrier'].mean(), 2),
    avg_quantity=int(data['quantity'].mean()),
    max_quantity=data['quantity'].max(),
    min_quantity=data['quantity'].min(),
    avg_sales=int(data['sales'].mean())
)
result.write()
print(result.check())

✅ Once your logic is encoded, commit and push your new file `seller.py`  
✅ Feel free to continue working on this notebook, or make a copy and delete the code above to start `section 2 - Exploration` from a fresh noteook

## 2 - Exploration

### 2.1 - Plots

👉 Let's start with some initial ***`EDA - Exploratory Data Analysis`*** about these sellers.

- 📈 Plot the distribution of each numerical variable of the dataset in one large figure
- 👀 Do you notice any outliers?
- What's the median of orders per seller ❓
- How does the distribution of this variable look like ❓

In [3]:
from olist.seller import Seller
seller = Seller()
sellers = seller.get_training_data()
sellers.head()

In [4]:
# YOUR CODE HERE

In [5]:
# YOUR CODE HERE

💡There seems to be a group of sellers which stands out for having very low review scores! 

📊 Let's investigate graphically it:
* Using `plotly`, create a `scatterplot` of `delay_to_carrier` against `wait_time`, varying bubble size by total `sales` for that seller, and coloring by `review_score`. 

In [6]:
# YOUR CODE HERE

Feel free to change values `x`, `y`, `color` and `size` to try identify who are the worst sellers

### 2.2 - Model out `review_score` with OLS

⚠️ Scatter plots have their limits. 

💡 A more rigorous way to explain **`sellers' review_score`** is to **model the impact of various features on `review_score` with a `multivariate-OLS` in `statsmodels`**.

👉 Create an OLS with numerical features of your choice. 

❓ What are the most impactful ones? 

⚠️ Don't forget to standardize your features using the `standardize`function below to compare the regression coefficients together. 

In [7]:
def standardize(df, features):
    df_standardized = df.copy()
    for f in features:
        mu = df[f].mean()
        sigma = df[f].std()
        df_standardized[f] = df[f].map(lambda x: (x - mu) / sigma)
    return df_standardized

In [8]:
# YOUR CODE HERE

📊 Draw a `bar_plot` with sorted coefficients.

In [10]:
# YOUR CODE HERE

👉 Finally, investigate your model's performance (`R-squared`) and `residuals`

In [11]:
# YOUR CODE HERE

👉 Compare the real review scores and the predicted scores by showing them on the same graph.

In [12]:
# YOUR CODE HERE

👉 Plot the residuals

In [13]:
# YOUR CODE HERE

### 2.3 - Add the `seller_state` to your analysis

❓ We haven't used information about `seller_state` yet.  
- Create a new OLS model regressing `review_score` on only on `seller_states` .
- Analyse your significant features using `return_significative_coef(model)` coded for you in `olist/utils.py`
- What are the best states in terms of `review_score`? 

<details>
    <summary>- Hints -</summary>
        
⚠️ Be careful, `seller_state` is a categorical feature. 
    
💡 Use `C(a_cat_feature)` in the formula to tell the linear regression model which variables are categorical variables. It will create one boolean variable `is_cat_feature_xx` **per unique category** 

</details>

In [14]:
# YOUR CODE HERE

☝️ Some states indeed have _significantly_ better reviews than others on average. 

🤔 Is it due to some lower `quantity_per_order`, lower `wait_time`, or `delay_to_carrier`?  Or is it due to some other factors that we haven't collected data about?

❓ Try to isolate the impact of the `seller_state` from the rest by adding other continuous features to your OLS until `seller_states` is no longer statistically siginificant!

In [37]:
# YOUR CODE HERE

In [16]:
# YOUR CODE HERE

☝️ After adding `wait_time` to our analysis, none of the 22 dummy variables `is_seller_state_xx` are statistically signigicant:

Given our small dataset (most states have a very limited number of sellers):
- We _cannot conclude_ that "some states are inherently better than other for reasons that would be independent of the `wait_time`" 
- In other words, we _cannot reject the hypothesis_ that "seller_state has no impact on review_score, other than through `wait_time`"

🏁 Congratulations!

💾 Commit and push :
* your ` sellers.ipynb`notebook 
* as well as `seller.py`